In [ ]:
from bs4 import BeautifulSoup
import requests
import warnings
import pandas as pd 
import urllib.parse
from lxml import html
import re
warnings.filterwarnings("ignore")

In [ ]:
url = "https://tunisie-medicale.com/index.php/docteur"

In [ ]:
response = requests.get(url,verify=False)

In [ ]:
response.status_code

200

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
soup #class="col-lg-6 col-md-6 col-sm-6 col-xs-12"

<!DOCTYPE html>

<html lang="fr">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<title>Liste de docteur / médecin  |  Tunisie Medical Annuaire</title>
<meta content="Annuaire spécialisé dans le domaine du médical, il permet de trouver un professionnel de santé, médecin, pharmacie, dentiste... en Tunisie" name="description"/>
<meta content="" name="keywords"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="contact@tunisie-medicale.com" name="author"/>
<link href="https://tunisie-medicale.com/favicon.ico" rel="icon" type="image/png"/>
<!-- Bootstrap Styles -->
<link href="https://tunisie-medicale.com/css/bootstrap.css" rel="stylesheet"/>
<!-- Awesome Icons -->
<link href="https://tunisie-medicale.com/css/font-awesome-5.3.1.css" rel="stylesheet"/>
<link href="https://tunisie-medicale.com/css/font-awesome.css" rel="stylesheet"/>
<!-- Carousel -->
<link href="https://tunisie-medicale.com/css/owl-carousel.css" rel="stylesh

In [ ]:
medecins = soup.find_all('div', {'class':'col-lg-6 col-md-6 col-sm-6 col-xs-12'})

In [ ]:
medecins[0]

<div class="col-lg-6 col-md-6 col-sm-6 col-xs-12">
<div class="team_widget">
<ul>
<li>
<img alt="harhira mohamed abderraouf - orl" class="img-responsive img-circle alignleft" src="https://tunisie-medicale.com/files/medecin/default.png"/>
<h2><a data-placement="top" data-toggle="tooltip" href="https://tunisie-medicale.com/index.php/docteur/3053-harhira-mohamed-abderraouf-orl-orl-et-stomatologie-jendouba" title="Harhira Mohamed Abderraouf - Orl">
<span>Dr. Harhira M…</span>
</a></h2>
<h3>ORL ET STOMAT…</h3>
<p class="doctor-details">
<span><i class="fa fa-map-marker"></i> Adresse : </span> <a href="http://maps.google.com/?q=36.501578938529015,8.785196368264208" target="_blank">Jendouba</a><br/>
                                        Jendouba Jendouba Sud<br/>
</p>
</li>
</ul>
</div><!-- end team_widget -->
</div>

In [ ]:
len(medecins)

20

In [ ]:
medecins[0].find('a', href=True).get("href")

'https://tunisie-medicale.com/index.php/docteur/3053-harhira-mohamed-abderraouf-orl-orl-et-stomatologie-jendouba'

In [ ]:
medecins[0].find_all('a', href=True)[1].get("href")

'http://maps.google.com/?q=36.501578938529015,8.785196368264208'

In [ ]:
one_med = "https://tunisie-medicale.com/index.php/docteur/3053-harhira-mohamed-abderraouf-orl-orl-et-stomatologie-jendouba"


In [ ]:
response = requests.get(one_med,verify=False)

In [ ]:
response.status_code

200

In [ ]:
soup_one_med = BeautifulSoup(response.content, 'html.parser')

In [ ]:
soup_one_med

In [ ]:
#div class="col-lg-8 col-md-8 col-sm-8 col-xs-12">
medecin = soup_one_med.find_all('div', {'class':'col-lg-8 col-md-8 col-sm-8 col-xs-12'})

In [ ]:
medecin

In [ ]:
name = medecin[0].find_all('span', {'itemprop':"name"})

In [ ]:
name[0].text

'Dr. Harhira Mohamed Abderraouf - Orl'

In [ ]:
#<h3 itemprop="medicalSpecialty"
speciality = medecin[0].find_all('h3',{'itemprop': 'medicalSpecialty'})

In [ ]:
speciality[0].text.lower()

'orl et stomatologie'

In [ ]:
map_link =  medecin[0].find_all('a',{'itemprop': 'streetAddress'})
#<a href="http://maps.google.com/?q=36.501578938529015,8.785196368264208" itemprop="streetAddress" target="_blank">Jendouba</a><br/>

In [ ]:
#map 
map_link[0].get('href')

'http://maps.google.com/?q=36.501578938529015,8.785196368264208'

In [ ]:
#adress
map_link[0].text

'Jendouba'

In [ ]:
#<p itemprop="addressLocality">Jendouba Jendouba Sud</p>
local =  medecin[0].find_all('p',{'itemprop': 'addressLocality'})[0].text


In [ ]:
local

'Jendouba Jendouba Sud'

In [ ]:
#tele 
#<a class="" href="tel:+21678605200" itemprop="telephone"> <span class="fa fa-phone"></span>  Appeler : (+216) 78 605 200</a> <br/>
#<p itemprop="addressLocality">Jendouba Jendouba Sud</p>
tele =  medecin[0].find_all('a',{'itemprop': 'telephone'})


In [ ]:
#telephone list 
t=[]
for i in tele:
  t.append(i.get('href'))


In [ ]:
#==> Automation
def extract_medecin_from_one_page(site):
  responses = requests.get(site,verify=False)
  soup = BeautifulSoup(responses.content, 'html.parser')
  results = soup.find_all('div', {'class':'col-lg-6 col-md-6 col-sm-6 col-xs-12'}) # ==> we have all info from one page
  liste = []
  for result in results:
    one_med_link = result.find('a', href=True).get("href")
    response = requests.get(one_med_link,verify=False)
    soup1 = BeautifulSoup(response.content, 'html.parser')
    medecin = soup1.find_all('div', {'class':'col-lg-8 col-md-8 col-sm-8 col-xs-12'})
    
    #get med infos
    info_med = {}
    if medecin != []:
      #get med name
      name = medecin[0].find_all('span', {'itemprop':"name"})
      info_med['name'] = name[0].text

      #get med speciality
      speciality = medecin[0].find_all('h3',{'itemprop': 'medicalSpecialty'})
      info_med['speciality'] = speciality[0].text.lower()
      
      #add med google maps link
      map_link =  medecin[0].find_all('a',{'itemprop': 'streetAddress'})
      info_med['map_link'] = map_link[0].get('href')
      
      #get med adress
      info_med['adress'] = map_link[0].text
      
      #get med gouvernant 
      info_med['local'] =  medecin[0].find_all('p',{'itemprop': 'addressLocality'})[0].text

      #get med phones numbers
      tele =  medecin[0].find_all('a',{'itemprop': 'telephone'})
      #telephone list 
      t=[]
      for i in tele:
        t.append(i.get('href'))
      info_med["telephones"] = t
      
    liste.append(info_med)

  return liste
  


In [ ]:
site = "https://tunisie-medicale.com/index.php/docteur/"
data = extract_medecin_from_one_page(site)

In [ ]:
data = pd.DataFrame(data)
data

In [ ]:
#Extract all data : scraping multiples pages
url = "https://tunisie-medicale.com/index.php/docteur/index/"
j=0 
for i in range(0,150):
  j=j+20
  data = pd.concat([data, pd.DataFrame(extract_medecin_from_one_page(url+str(j)))])

In [ ]:
data.head(n=2)

,name,speciality,map_link,adress,local,telephones
0,Dr. Harhira Mohamed Abderraouf - Orl,orl et stomatologie,"http://maps.google.com/?q=36.501578938529015,8...",Jendouba,Jendouba Jendouba Sud,"[tel:+21678605200, tel:+21695343403]"
1,Dr. Houcine Mnasri,anesthesie-reanimation,"http://maps.google.com/?q=37.2872637875397,9.8...",153 avenue habib bourguiba- hippocenter- BIZERTE,Ariana Ariana Ville,"[tel:+21698714000, tel:+21672434444]"


In [ ]:
data= data.reset_index()

In [ ]:
data.to_csv("siwar.csv")

In [ ]:
data.to_excel("siwar.xlsx")  

In [ ]:
#good job